In [4]:
import os
import pickle

pkl_path = "/teamspace/studios/this_studio/modal_remote/ScoreFlow/runs/Qwen-Qwen2.5-7B-Instruct_MATH_0/Qwen-Qwen2.5-7B-Instruct_MATH_0.pkl"

if not os.path.exists(pkl_path):
    raise FileNotFoundError(f"Không tìm thấy file {pkl_path}")
with open(pkl_path, "rb") as f:
    data = pickle.load(f)

len(data)

952

In [9]:
ref_prob = "A particular convex pentagon has two congruent, acute angles. The measure of each of the other interior angles is equal to the sum of the measures of the two acute angles. What is the common measure of the large angles, in degrees?"

In [11]:
graph_num = 8  # number of graph variants per problem

# Assume `data` is already loaded (from the pickle)
print(f"Total records: {len(data)}")
num_problems = len(data) // graph_num
print(f"Expecting {num_problems} unique problems (8 graphs each)\n")

error_groups = []

for group_idx in range(num_problems):
    start = group_idx * graph_num
    end = start + graph_num
    group = data[start:end]
    
    # Reference problem text (normalize whitespace)
    ref_prob = group[0][0]["problem"].strip()
    all_same = all(
        (item[0]["problem"].strip() == ref_prob)
        for item in group
    )
    
    if not all_same:
        print(f"❌ Mismatch found in group {group_idx} (indices {start}-{end-1})")
        for idx, item in enumerate(group, start=start):
            print(f"  idx={idx} → {item[0]['problem'][:80]!r}")
        error_groups.append(group_idx)

if not error_groups:
    print("\n✅ All problems are grouped correctly (8 per problem, contiguous).")
else:
    print(f"\n⚠️ Found {len(error_groups)} groups with mismatched problems: {error_groups}")


Total records: 952
Expecting 119 unique problems (8 graphs each)


✅ All problems are grouped correctly (8 per problem, contiguous).


In [1]:
import json

with open("runs/Qwen-Qwen2.5-7B-Instruct_MATH_0/Qwen-Qwen2.5-7B-Instruct_MATH_0.json", "r") as f:
    workflow_data = json.load(f)

len(workflow_data)

952

In [2]:
with open("part_1.json", "r") as f:
    part1_eval = json.load(f)

len(part1_eval)

550

In [3]:
with open("part_1.json", "r") as f:
    part1_eval = json.load(f)

len(part1_eval)

with open("runs/Qwen-Qwen2.5-7B-Instruct_MATH_0/Qwen-Qwen2.5-7B-Instruct_MATH_0_scores_0_named.json", "r") as f:
    part2_eval = json.load(f)

len(part2_eval)

456

In [4]:
print(f"Part 1 entries: {len(part1_eval)}")
print(f"Part 2 entries: {len(part2_eval)}")

# === Merge while removing duplicates ===
merged = []
seen_keys = set()

def key(entry):
    return (entry["question_id"], entry["graph_id"], entry["rep_id"])

for entry in part1_eval + part2_eval:
    k = key(entry)
    if k not in seen_keys:
        merged.append(entry)
        seen_keys.add(k)

print(f"✅ Merged entries: {len(merged)} (removed {len(part1_eval) + len(part2_eval) - len(merged)} duplicates)")

# === Save result ===
output_file = "merged_eval.json"
# with open(output_file, "w") as f:
#     json.dump(merged, f, indent=2, ensure_ascii=False)

print(f"💾 Saved merged file → {output_file}")

Part 1 entries: 550
Part 2 entries: 456
✅ Merged entries: 952 (removed 54 duplicates)
💾 Saved merged file → merged_eval.json


In [6]:
temp_data = []

for item in workflow_data:
    cur = {
        "problem": item[0]["problem"],
        "level": item[0]["level"],
        "type": item[0]["type"],
        "solution": item[0]["solution"],
        "graph": item[1]
    }
    temp_data.append(cur)

temp_data[0]

{'problem': 'A particular convex pentagon has two congruent, acute angles. The measure of each of the other interior angles is equal to the sum of the measures of the two acute angles. What is the common measure of the large angles, in degrees?',
 'level': 'Level 5',
 'type': 'Prealgebra',
 'solution': 'If $x$ is the measure in degrees of each of the acute angles, then each of the larger angles measures $2x$ degrees.  Since the number of degrees in the sum of the interior angles of an $n$-gon is $180(n-2)$, we have \\[\nx+x+2x+2x+2x=540 \\implies 8x = 540 \\implies x=135/2.\n\\] The large angles each measure $2x=\\boxed{135}$ degrees.',
 'graph': ' <graph>\nclass Workflow:\n    def __init__(\n        self,\n        config,\n        problem\n    ) -> None:\n        self.problem = problem\n        self.config = create(config)\n        self.gid = None\n        self.custom = operator.Custom(self.config, self.gid, self.problem)\n        self.sc_ensemble = operator.ScEnsemble(self.config, se

In [9]:
for item in merged:
    item["problem"] = temp_data[item["question_id"] + item["graph_id"]]["problem"]
    item["level"] = temp_data[item["question_id"] + item["graph_id"]]["level"]
    item["type"] = temp_data[item["question_id"] + item["graph_id"]]["type"]
    item["solution"] = temp_data[item["question_id"] + item["graph_id"]]["solution"]

    break

merged[0]

{'question_id': 0,
 'graph_id': 3,
 'rep_id': 0,
 'score': 0.0,
 'prompt': 'A particular convex pentagon has two congruent, acute angles. The measure of each of the other interior angles is equal to the sum of the measures of the two acute angles. What is the common measure of the large angles, in degrees?',
 'pred_code': "1 validation error for ReviewOp_AN\n  Value error, Missing fields: {'revised_solution'} [type=value_error, input_value={'thought': '1. **Logical...f the angles involved.'}, input_type=dict]\n    For further information visit https://errors.pydantic.dev/2.9/v/value_error",
 'log': 'If $x$ is the measure in degrees of each of the acute angles, then each of the larger angles measures $2x$ degrees.  Since the number of degrees in the sum of the interior angles of an $n$-gon is $180(n-2)$, we have \\[\nx+x+2x+2x+2x=540 \\implies 8x = 540 \\implies x=135/2.\n\\] The large angles each measure $2x=\\boxed{135}$ degrees.',
 'problem': 'A particular convex pentagon has two con

In [10]:
with open(output_file, "w") as f:
    json.dump(merged, f, indent=2, ensure_ascii=False)